In [ ]:
objects=[]



{'Array': <tecton_gen_ai.testing.examples.copilot._FuncOrClass>,
 'Field': <tecton_gen_ai.testing.examples.copilot._FuncOrClass>,
 'Map': <tecton_gen_ai.testing.examples.copilot._FuncOrClass>,
 'SdkDataType': <tecton_gen_ai.testing.examples.copilot._FuncOrClass>,
 'StrictFrozenModel': <tecton_gen_ai.testing.examples.copilot._FuncOrClass>,
 'Struct': <tecton_gen_ai.testing.examples.copilot._FuncOrClass>,
 'TectonValidationError': <tecton_gen_ai.testing.examples.copilot._FuncOrClass>}

In [ ]:
import inspect
from typing import Union, get_type_hints
from tecton_gen_ai.testing.examples.copilot import _FuncOrClass
import enum

pool = {}

def get_full_name(obj):
    return obj.__module__ + "." + obj.__name__

def is_tecton_type(tp):
    try:
        tp_str = tp.__module__
        return tp_str.startswith("tecton.") or tp_str=="tecton"
    except Exception:
        return False
    
def is_abdstract(cls):
    return inspect.isabstract(cls)

def find_tecton_parents(cls):
    for parent in cls.__bases__:
        if is_tecton_type(parent):
            yield parent
        


def find_tecton_annotations(cls):
    try:
        if inspect.isclass(cls):
            annotations = get_type_hints(cls.__init__)
        elif inspect.isfunction(cls):
            annotations = get_type_hints(cls)
        else:
            raise ValueError("cls must be a class or a function")
        for name, param in annotations.items():
            if is_tecton_type(param):
                yield param
            # else check if param is union, check if any of the union is a tecton type
            elif hasattr(param, "__origin__") and param.__origin__ == Union:
                for arg in param.__args__:
                    if is_tecton_type(arg):
                        yield arg
    except Exception:
        return None

def find_tecton_mentions(obj):
    # check obj.obj is a class type use inspect
    if inspect.isclass(obj):
        if not issubclass(obj, enum.Enum):
            yield from find_tecton_annotations(obj)
    elif inspect.isfunction(obj):
        yield from find_tecton_annotations(obj)

def find_tecton_dependencies(scope):
    res = {}
    for key, obj in scope.items():
        res[key]= list(x.__name__ for x in find_tecton_mentions(obj.obj) if x.__name__ in scope)
    return res

def build_api_graph():
    objects = list(_FuncOrClass.from_expressions(["tecton"]))
    scope = {x.name: x for x in objects}
    deps = find_tecton_dependencies(scope)
    res = {}
    for key, value in scope.items():
        res[key] = {"declaration": value.callable_declaration, "deps": deps[key]}
    return res

def build_code(name, graph, code):
    if name in code:
        return
    for dep in graph[name]["deps"]:
        build_code(dep, graph, code)
    code[name] = graph[name]["declaration"]


graph = build_api_graph()

code = {}
build_code("BatchFeatureView", graph, code)
print("\n\n".join(code.values()))

class BatchTriggerType(enum.Enum):
    SCHEDULED = feature_view__args_pb2.BatchTriggerType.BATCH_TRIGGER_TYPE_SCHEDULED
    MANUAL = feature_view__args_pb2.BatchTriggerType.BATCH_TRIGGER_TYPE_MANUAL
    NO_BATCH_MATERIALIZATION = feature_view__args_pb2.BatchTriggerType.BATCH_TRIGGER_TYPE_NO_BATCH_MATERIALIZATION

class DatabricksClusterConfig(StrictModel):
    """Configuration used to specify materialization cluster options on Databricks.

    This class describes the attributes of the new clusters which are created in Databricks during
    materialization jobs. You can configure options of these clusters, like cluster size and extra pip dependencies.

    Note on `extra_pip_dependencies`: This is a list of packages that will be installed during materialization.
    To use PyPI packages, specify the package name and optionally the version, e.g. `"tensorflow"` or `"tensorflow==2.2.0"`.
    To use custom code, package it as a Python wheel or egg file in S3 or DBFS, then specify the path 

In [ ]:
import json

all_class_names = sorted(set([x.__name__ for x in find_tecton_mentions(objects)]))
all_class_names = json.dumps(all_class_names)
all_class_names

tecton.framework.feature.Aggregate
<class 'tecton.aggregation_functions.AggregationFunction'>
<class 'tecton.framework.configs.TimeWindow'>
<class 'tecton.framework.configs.TimeWindowSeries'>
<class 'tecton.framework.configs.LifetimeWindow'>
tecton.framework.feature_view.AggregationLeadingEdge
tecton.framework.feature.Attribute
tecton.framework.configs.AutoscalingConfig
tecton.framework.feature_view.BatchFeatureView
tecton.framework.data_source.BatchSource
tecton.framework.feature_view.BatchTriggerType
tecton.framework.configs.BigQueryConfig
<class 'tecton.framework.utils.Secret'>
tecton.framework.configs.BigtableConfig
tecton.framework.configs.CacheConfig
tecton.framework.feature.Calculation
tecton_core.compute_mode.ComputeMode
tecton.framework.data_frame.TectonDataFrame
tecton.framework.data_source.DataSource
tecton.framework.configs.DatabricksClusterConfig
tecton.framework.configs.DatabricksJsonClusterConfig
tecton.framework.configs.DataprocJsonClusterConfig
tecton.framework.dataset

'["Aggregate", "AggregationFunction", "AggregationLeadingEdge", "Attribute", "AutoscalingConfig", "BatchFeatureView", "BatchSource", "BatchTriggerType", "BigQueryConfig", "BigtableConfig", "CacheConfig", "Calculation", "DataSource", "DatabricksClusterConfig", "DatabricksJsonClusterConfig", "DataprocJsonClusterConfig", "Dataset", "DatetimePartitionColumn", "DeltaConfig", "DynamoConfig", "EMRClusterConfig", "EMRJsonClusterConfig", "Embedding", "Entity", "FeatureMetadata", "FeatureReference", "FeatureServerGroup", "FeatureService", "FeatureTable", "FeatureVector", "FeatureView", "Field", "FileConfig", "HiveConfig", "Inference", "KafkaConfig", "KafkaOutputStream", "KinesisConfig", "KinesisOutputStream", "LifetimeWindow", "LoggingConfig", "MaterializationAttempt", "MaterializationJob", "MaterializedFeatureView", "ModelConfig", "OfflineStoreConfig", "PandasBatchConfig", "PandasDataSourceFunctionType1", "PandasDataSourceFunctionType2", "PandasDataSourceFunctionType3", "PandasDataSourceFunctio

In [ ]:
from tecton_gen_ai.testing import set_dev_mode
from tecton_gen_ai.api import Agent
from pydantic import BaseModel, Field

class Output(BaseModel):
    code: str = Field(..., description="The generated python code")

set_dev_mode()

agent = Agent(
    name = "parser",
    prompt = """
You are given a declaration of a class or a function, convert it to a pydantic model representation
You only need to output the pydantic model code, no import needed
You need to make sure each field is defined using pydantic Field, but you should use `PField` instead

parameters docstrings should not be included in the final class docstring

For example

```python
class SomeView:
    '''This is a class docstring for SomeView'''

    def __init__(self, a: int, b: Entity):
        '''
        more explanations 1

        :param a: parameter a
        :param b: parameter b

        more explanations 2
        '''
        pass
```

should be translated to

```python
class SomeView(BaseModel):
    '''This is a class docstring for SomeView
    
    more explanations 1

    more explanations 2
    '''

    a: int = PField(..., description="parameter a")
    b: Entity = PField(..., description="parameter b")
```

The output should just be the python code without backticks

""",
    llm = {"model": "openai/gpt-4o-2024-11-20", "temperature": 0.0},
    output_schema=Output,
)

In [ ]:
import tqdm


code = []

for obj in tqdm.tqdm(objects[:5]):
    if inspect.isclass(obj.obj):
        res = agent.invoke(obj.declaration)
        code.append(res["code"])

  0%|          | 0/5 [00:00<?, ?it/s]

100%|██████████| 5/5 [00:28<00:00,  5.62s/it]


In [ ]:
print("\n\n".join(code))

class Aggregate(BaseModel):
    '''The `Aggregate` class describes an aggregation feature that is applied to a Batch or Stream Feature View via `features` param.

    ```python
    from tecton import Aggregate, batch_feature_view, TimeWindow
    from tecton.types import Int64
    from datetime import timedelta

    @batch_feature_view(
        # ...
        features=[
            Aggregate(
                input_column=Field("my_column", Int64),
                function="mean",
                time_window=TimeWindow(window_size=timedelta(days=7)),
            ),
            Aggregate(
                input_column=Field("another_column", Int64),
                function="mean",
                time_window=TimeWindow(window_size=timedelta(days=1)),
                name="1d_average",
                description="my aggregate feature description",
                tags={"tag": "value"}
            ),
        ],
    )
    def my_fv(data_source):
        pass
    ```
    '''

    function: Ag